# INTRODUCTION

It this project, the following dataset of [used cars from eBay Kleinanzeigen](https://data.world/data-society/used-cars-data), a classifieds section of the German eBay website, will be used.

### Table of Contents <a class="anchor" id="s0"></a>

* [Dataset and aim of the study](#s1)
* [Data cleaning and exploration](#s2)
* [Odometer investigation](#s3)
* [Registration year investigation](#s4)
* [Exploring Price by Brand](#s5)
* [Correlation between price and mileage (odometer_km)?](#s6)
* [Further data cleaning](#s7)
* [Most common brand/model combinations](#s8)
* [How much cheaper are cars with damage than their non-damaged counterparts?](#s9)
    

## Dataset and aim of the study <a class="anchor" id="s1"></a>

<span style='background :yellow' > Few modifications were made from the original dataset:</span>
- Sampled down to 50,000 data points to ensure the code runs quickly
- Modified to bring it closer to a real scraped dataset

---
--- 

The data dictionary provided with the dataset is as follows:

| Column name | Description |
| :--- | :--- |
|dateCrawled| When this ad was first crawled. All field-values are taken from this date.|
|seller| Whether the seller is private or a dealer.|
|offerType | The type of listing.| 
|price| The price on the ad to sell the car.|
|abtest| Whether the listing is included in an A/B test.|
|vehicleType| The vehicle Type.|
|yearOfRegistration| The year in which the car was first registered.|
|gearbox| The transmission type.|
|powerPS| The power of the car in PS.|
|model| The car model name.|
|kilometer| How many kilometers the car has driven.|
|monthOfRegistration| The month in which the car was first registered.|
|fuelType| What type of fuel the car uses.|
|brand| The brand of the car.|
|notRepairedDamage| If the car has a damage which is not yet repaired.|
|dateCreated| The date on which the eBay listing was created.|
|nrOfPictures| The number of pictures in the ad.|
|postalCode| The postal code for the location of the vehicle.|
|lastSeenOnline| When the crawler saw this ad last online.|

---

### <span style='color:Blue'> The aim of this project is to clean the data and analyze the included used car listings.<span>

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

FileNotFoundError: [Errno 2] No such file or directory: 'autos.csv'

In [ ]:
autos

In [ ]:
autos.info()

We can see that a lot of data columns are object data type despit it should be integer or float, such as:
- dateCrawled
- price                
- odometer             
- dateCreated          
- lastSeen             

# Data cleaning and exploration <a class="anchor" id="s2"></a>

The value of rangeindex provided for the index axis indicates **371528** entries.

The summary includes list of all columns with their data types, most of which are strings, and the number of non-null values in each column. 

The following columns have null values, all with string values:
- vehicleType 
- gearbox
- model
- fuelType
- notRepairedDamage

Note that column names use camelcase instead of Python's preferred snakecase. 


In [ ]:
new_columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'powerPS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

autos.columns = new_columns

print(autos.columns)

The column names were converted from camelcase to snakecase and some of the column names were reworded based on the data dictionary to be more descriptive.

In [ ]:
autos.gearbox.value_counts()

Looking at the data, we can see that <span style='background:pink'>price</span> and <span style='background:pink'>odometer</span> columns are numeric values stored as text. 

For each column:
- Any non-numeric character will be removed;
- The column will be converted to a numeric dtype.

In [ ]:
autos["price"] = autos["price"].str.replace('$','')
autos["price"] = autos["price"].str.replace(',','')
autos["price"] = autos["price"].astype(int)
autos["price"].head()

In [ ]:
autos["odometer"] = autos["odometer"].str.replace('km','')
autos["odometer"] = autos["odometer"].str.replace(',','')
autos["odometer"] = autos["odometer"].astype(int)
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

In [ ]:
autos["odometer_km"].head()

Now, let's check these column to look for any values that look unrealistically high or low (outliers) that might be good to remove.

## Odometer investigation <a class="anchor" id="s3"></a>
---

In [ ]:
# how many unique values?
autos["odometer_km"].unique().shape

In [ ]:
# Statistics
autos["odometer_km"].describe()

In [ ]:
# Value count in ascending order
autos["odometer_km"].value_counts().sort_index(ascending=True)

In [ ]:
autos[(autos["odometer_km"]==150000)].sort_index(ascending=False).head(5)

Looking at the price and brand car, the data for the **odometer_km** column are coherent and are not outliers. A quick look at this [website](https://www.autoscout24.com/lst/opel/vectra) confirms this hypothesis.

## Price investigation <a class="anchor" id="s4"></a>
---

In [ ]:
# how many unique values?
autos["price"].unique().shape

In [ ]:
# Statistics
autos["price"].describe()

In [ ]:
# Value count in descending order
autos["price"].value_counts().sort_index(ascending=False).head(15)

A quick look at the price value count shows us <span style='background :yellow'> **OUTLIERS** with price > 350 000 $ </span>.

Let's confirm by looking closely at the most expansive used car of the database if it "makes sense".

In [ ]:
autos[autos["price"]>=350000].sort_values(by='price', ascending=True).head()

A quick look at this [website](https://www.reezocar.com/en/) shows **Porsh 991** price around 345,000 \\$  for 5000 km used. But a **Volkswagen Jetta Gt** from 1989 with 280,000 km used is sold at 2,230 \\$ approximately. This confirms OUTLIERS with price > 350,000 \\$

In [ ]:
# Value count in ascending order
autos["price"].value_counts().sort_index(ascending=True).head()

In [ ]:
autos[autos["price"] == 0].head(10)

A price of 0\\$ usually indicated a price on demand. A quick look at this [website](https://www.reezocar.com/en/) shows **Porsche Targa 911 S** with a price on demand. If we check used **Porsche 911** for the years arounf 1970 and 5000 km used, prices are superior to 80,000 \\$. So we might want to exclude the price on demand, hence the rows with a price == 0\\$.

In [ ]:
clean_autos=autos[(autos["price"] >= 1) & (autos["price"] <= 350000)]

## Registration year investigation <a class="anchor" id="s4"></a>
---

The <span style='background:pink'>date_crawled</span>, <span style='background:pink'>last_seen</span> and <span style='background:pink'>ad_created</span> columns are all identified as string values. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values.

In [ ]:
clean_autos[['date_crawled','ad_created','last_seen']][0:5]

In [ ]:
# dropna = 'True' will include missing values, and normalize = true will use percentages instead of counts
print("First added by the crawler: \n")
df1=clean_autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
print(df1)

In [ ]:
print("\nWhen the crawler saw this ad last online: \n")
df2=clean_autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
print(df2)

In [ ]:
print("\nThe date on which the eBay listing was created.\n")
df3=clean_autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
print(df3)

We can see there are inaccuracies between `date_crawled` or `ad_created` for any car with a `registration_year` above 2016. Indeed, a car cannot be first registered after the eBay listing.

In [ ]:
clean_autos['registration_year'].describe()

The minimum and maximum values indicate odd values: 
- `min` = 1000, which is inferior to the [year 1886 is regarded as the birth year of the modern car](https://en.wikipedia.org/wiki/Car#:~:text=The%20year%201886%20is%20regarded,by%20the%20Ford%20Motor%20Company.)
- `max` = 9999, which is far superior to the current year 

In [ ]:
clean_autos['registration_year'].value_counts().sort_index().head(10)

A quick search on internet indicates cars were invented in the late 1800s. Let's see what kind of cars are listed for the year `1910` and `1927`.

In [ ]:
clean_autos[(clean_autos['registration_year']==1910) | (clean_autos['registration_year']==1927)].sort_values(by='registration_year', ascending=True)

A quick search on internet indicates Renault Twingo first generation was in 1993. But the [Essex super six Ford]('https://www.conceptcarz.com/vehicle/z11897/essex-super-six.aspx') is a car from the year `1927` which is the lowest acceptable registration year of the dataset.

In [ ]:
clean_autos['registration_year'].value_counts().sort_index().tail(10)

Let's remove the values outside the 1927 - 2016 interval.

In [ ]:
clean_autos=clean_autos[(clean_autos['registration_year'] >=1927) & (clean_autos['registration_year'] <=2016)]
clean_autos['registration_year'].describe()

In [ ]:
clean_autos['registration_year'].value_counts(normalize=True).head(10).sort_index()

Cars with a registration year between the `1998-2007` interval are the most listed on eBay. `2000`being the registration year with most listed cars.

## Exploring Price by Brand <a class="anchor" id="s5"></a>

In [ ]:
clean_autos['brand'].value_counts(normalize=True, sort=True)

Let's explore variations across different car brands in terms of __price__. In order to do so, we will aggregate over the top 20 brands using `Series.index` attribute to access the labels.

In [ ]:
selected_brands= clean_autos['brand'].value_counts(normalize=True, sort=True).index[:20]
selected_brands

In [ ]:
brand_mean_price = {}

for b in selected_brands:
    sel_brand = clean_autos[clean_autos['brand']== b]
    brand_mean_price[b] = sel_brand['price'].mean().round()
    
brand_mean_price_sorted=sorted(brand_mean_price.items(), key=lambda item: item[1], reverse = True)

In [ ]:
# Use .from_records() method to creates a DataFrame object from a structured ndarray
pd.DataFrame.from_records(brand_mean_price_sorted, columns=["Brand", "Price"])

As we can see, __sonstige__ and __mini__ cars are the most expansive cars listed on eBay with an average price superior to 10k\\$. __Audi__ cars are not far behind with an average price superior to 9k\\$  The least expansive listed cars on eBay are __opel__, __fiat__ and __renault__ cars with an average price under 3k\\$. Let's use aggregation to understand the average mileage for those cars and see if there's any visible link with mean price.

## Correlation between price and mileage (odometer_km)? <a class="anchor" id="s6"></a>

In [ ]:
brand_mean_odometer = {}

for b in selected_brands:
    sel_brand = clean_autos[clean_autos['brand']== b]
    brand_mean_odometer[b] = sel_brand['odometer_km'].mean().round()

# Convert both dictionaries to series objects, using the series constructor
bmp_series= pd.Series(brand_mean_price)
bmo_series= pd.Series(brand_mean_odometer)

#Create a dataframe from the first series object using the dataframe constructor
df = pd.DataFrame(bmp_series, columns=['mean_price'])
df['mean_odometer_km'] = bmo_series
df.sort_values('mean_price', ascending=False)

As we can see, __sonstige__ and __mini__ cars stand out a little bit with an average mileage lower than the rest of the cars. Unfortunately, it is difficult to judge the impact of mileage among the different car brands as such. Let's create a subset of the dataframe for __Upper middle class cars__. Mileage will be grouped into 3 bins in order to have a wider sens of its influence on the average price.

In [ ]:
upper_middle_class = clean_autos[(clean_autos['brand']== 'sonstige_autos')|(clean_autos['brand']== 'mini')|(clean_autos['brand']== 'audi')|(clean_autos['brand']== 'mercedes_benz')|(clean_autos['brand']== 'bmw')]
print("The 3 intervals of mileage for upper middle class cars:")
upper_middle_class['odometer_km'].value_counts(bins=3)

In [ ]:
# Mileage group 1 (101666.667, 150000.0]
price1=upper_middle_class.loc[upper_middle_class['odometer_km'] >= 101666.667, 'price'].mean().round()
print("Average price for cars with mileage range from 101666.667 to 150000.0 kms: ", price1, "$")

# Mileage group2 (53333.333, 101666.667]
price2=upper_middle_class.loc[(upper_middle_class['odometer_km'] > 53333.333) & (upper_middle_class['odometer_km'] <101666.667), 'price'].mean().round()
print("Average price for cars with mileage range from 53333.333 to 101666.667 kms: ", price2, "$")

# Mileage group 3 (4854.999, 53333.333]
price3=upper_middle_class.loc[upper_middle_class['odometer_km'] <= 53333.333, 'price'].mean().round()
print("Average price for cars with mileage range from 4854.999 to 53333.333 kms: ", price3, "$")

### Here are some conclusion we can draw from these results:
- In general, brand reputation affects more the price listed than recorded mileage
- For upper middle class cars, price tends to be less expansive with higher mileage

## Further data cleaning <a class="anchor" id="s7"></a>

As further data cleaning, something we could do is to identify categorical data that uses german words to translate them and map the calues to their english counterparts.

As we saw in the [Dataset and aim of the study](#s1), the following columns have the dtypes `object`:

In [ ]:
categorical_data = clean_autos.select_dtypes(include='object')
categorical_data.info()

Let's remove all the columns related with dates or to the properties of the car such as the columns __model__ and __brand__. Let's use the pandas function `.unique` to verify what kind of **unique** value each column returns and if they are in German and translate them in english using Google Translate and finally using the pandas function `.replace` to map the values to their english counterparts.

In [ ]:
print("Unique value for 'seller' column - before replace : \n", categorical_data['seller'].unique())
clean_autos['seller'].replace({"gewerblich":"commercial"},inplace = True)
print("\nUnique value for 'seller' column - after replace : \n", clean_autos['seller'].unique())

In [ ]:
print("Unique value for 'offer_type' column - before replace : \n", categorical_data['offer_type'].unique())
clean_autos['offer_type'].replace({"Angebot":"offer"},inplace = True)
print("\nUnique value for 'offer_type' column - after replace :\n", clean_autos['offer_type'].unique())

In [ ]:
# no German words to translate
categorical_data['abtest'].unique()

In [ ]:
# to return unique values without the NaN, simply chain the dropna and unique functions together
print("Unique value for 'vehicle_type' column - before replace : \n", categorical_data['vehicle_type'].dropna().unique())
clean_autos['vehicle_type'].replace({"kleinwagen":"small car", "kombi":"combi", "andere":"other"},inplace = True)
print("\nUnique value for 'vehicle_type' column - after replace : \n", clean_autos['vehicle_type'].dropna().unique())

In [ ]:
print("Unique value for 'gearbox' column - before replace : \n", categorical_data['gearbox'].dropna().unique())
clean_autos['gearbox'].replace({"manuell":"manually", "automatik":"automatic"},inplace = True)
print("\nUnique value for 'gearbox' column - after replace : \n", clean_autos['gearbox'].dropna().unique())

In [ ]:
print("Unique value for 'fuel_type' column - before replace : \n", categorical_data['fuel_type'].dropna().unique())
clean_autos['fuel_type'].replace({"benzin":"petrol", "elektro":"electro", "andere":"other"},inplace = True)
print("\nUnique value for 'fuel_type' column - after replace : \n", clean_autos['fuel_type'].dropna().unique())

In [ ]:
print("Unique value for 'unrepaired_damage' column - before replace : \n", categorical_data['unrepaired_damage'].dropna().unique())
clean_autos['unrepaired_damage'].replace({"nein":"No", "ja":"Yes"},inplace = True)
print("\nUnique value for 'unrepaired_damage' column - after replace : \n", clean_autos['unrepaired_damage'].dropna().unique())

## Most common brand/model combinations <a class="anchor" id="s8"></a>

In [ ]:
# Use .size() property to get an int representing the number of elements in this object
print("Top 5 Most common brand/model combinations :\n \n", clean_autos.groupby(['brand', 'model']).size().sort_values(ascending=False).head())

## How much cheaper are cars with damage than their non-damaged counterparts? <a class="anchor" id="s9"></a>

Let's explore variations across cars with damage than their non-damaged counterparts in terms of price. In order to do so, we will aggregate over the top 5 brands using Series.index attribute to access the labels. 

In [ ]:
# Select upper middle class cars
clean_upper_middle_class=clean_autos.loc[(clean_autos['brand']== 'sonstige_autos')|(clean_autos['brand']== 'mini')|(clean_autos['brand']== 'audi')|(clean_autos['brand']== 'mercedes_benz')|(clean_autos['brand']== 'bmw')]

# group by 'unrepaired_damage' & 'brand' and select 'price' column
df_damage = clean_upper_middle_class.groupby(['unrepaired_damage', 'brand'], as_index=False).price.mean().round()

# unrepaired_damage= df_damage[df_damage['unrepaired_damage']=='No'].sort_values(by = 'price', ascending=False)
# repaired_damage= df_damage[df_damage['unrepaired_damage']=='Yes'].sort_values(by = 'price', ascending=False)

unrepaired_damage= df_damage[df_damage['unrepaired_damage']=='No']
repaired_damage= df_damage[df_damage['unrepaired_damage']=='Yes']
display(unrepaired_damage)
display(repaired_damage)

In [ ]:
# Reset index for comparison
unrepaired_damage.reset_index(inplace=True)
repaired_damage.reset_index(inplace=True)

# Extract "price" column for unrepaired_damage cars 
unrepaired_price=unrepaired_damage.loc[:,'price']

# Extract "price" column for repaired_damage cars 
repaired_price=repaired_damage.loc[:,'price']

# mean difference between the 2 "price" columns
price_damage_comparison=unrepaired_damage['price'] - repaired_damage['price']

# mean percentage difference between the 2 "price" columns
price_damage_percent_diff=((unrepaired_damage['price'] - repaired_damage['price'])/repaired_damage['price']).round()

#Create a dataframe from the first series object using the dataframe constructor
df_damage = pd.DataFrame(unrepaired_damage.loc[:,'brand'], columns=['brand'])
df_damage['price_mean_diff'] = price_damage_comparison
df_damage.sort_values('price_mean_diff', ascending=False)
df_damage['price_diff (%)'] = price_damage_percent_diff
df_damage

Unsurprisingly, the cars with repaired damages are more expansive that their non-repaired counterparts. 

[![alt text](https://img.webnots.com/2016/01/arrow53.png)](#s0)